<a href="https://colab.research.google.com/github/NckolasGomes6696/IPet/blob/main/Mod_3_imagens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Face and Eye Recognition
Before we discuss redaction, lets cover the basics of identifying a face within an image.

Within OpenCV-Python, we can use the cv2.rectangle method to draw a rectangle over an image. We need five parameters for this function: image, start_point, end_point, color, and thickness.

We can get the start and end points by running a classier to identify the area that contains the face. The code block below uses the haarcascade classifier.  Images must be converted to grayscale before applying the classifier.  The end result is the detectmultiscale function returns an array that correspond to the coordinates for the rectangle that you will input in the opencv rectangle function.

A similar pattern is performed to identify any other features such as the eyes which is done in the code block below. We draw the rectangles for the face and eye over the original image and display it in a widget.


In [11]:
#https://www.geeksforgeeks.org/how-to-blur-faces-in-images-using-opencv-in-python/
#https://docs.opencv.org/3.1.0/d7/d8b/tutorial_py_face_detection.html#gsc.tab=0
# Importing libraries
import numpy as np
import cv2
import ipywidgets
from IPython.display import display
#from google.colab.patches import cv2_imshow

# A function for plotting the images

oimage_widget = ipywidgets.Image(format='png')
image_widget = ipywidgets.Image(format='png')
# Reading an image using OpenCV
# OpenCV reads images by default in BGR format
image = cv2.imread('/content/lena_color.png')

# Converting BGR image into a RGB image
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# plotting the original image
oimage = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
oimage_widget.value = bytes(cv2.imencode('.png', oimage)[1])
display(oimage_widget)

#need to ensure this matches the actual file locations!
fpath='/content/cascades/haarcascade_frontalface_alt.xml'
epath='/content/cascades/haarcascade_eye.xml'

face_detect = cv2.CascadeClassifier(fpath)
eye_detect = cv2.CascadeClassifier(epath)

#returns an array that identifies the four points that contain the face
faces = face_detect.detectMultiScale(gray, 1.3, 5)
for (x,y,w,h) in faces:
    cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,0),2)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = image[y:y+h, x:x+w]
    eyes = eye_detect.detectMultiScale(roi_gray)
    for (ex,ey,ew,eh) in eyes:
        cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)


# Display the output
image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
image_widget.value = bytes(cv2.imencode('.png', image)[1])
display(image_widget)


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x00\x00\x00\x01\x00\x08\x02\x00\x00\x00\xd3\x10?…

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x00\x00\x00\x01\x00\x08\x02\x00\x00\x00\xd3\x10?…

### Redact an image - CPU
Now lets blur the are within the rectangle to redact the face using the OpenCV GaussianBlur function. We will not identify eyes in this code block, just the face.

In [14]:
%%time
import numpy as np
import cv2
import ipywidgets
from IPython.display import display
import numpy as np

# Reading an image using OpenCV
image = cv2.imread('/content/lena_color.png')
image_widget = ipywidgets.Image(format='png')

# Converting BGR image into a RGB image
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

fpath='/content/cascades/haarcascade_frontalface_alt.xml'

face_detect = cv2.CascadeClassifier(fpath)

faces = face_detect.detectMultiScale(gray, 1.3, 5)
for (x,y,w,h) in faces:
    cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,0),2)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = image[y:y+h, x:x+w]
    roi = cv2.GaussianBlur(roi_color, (23, 23), 30)
    # impose this blurred image on original image to get final image
    image[y:y+roi.shape[0], x:x+roi.shape[1]] = roi

# Display the output
image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
image_widget.value = bytes(cv2.imencode('.png', image)[1])
display(image_widget)


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x00\x00\x00\x01\x00\x08\x02\x00\x00\x00\xd3\x10?…

CPU times: user 85.9 ms, sys: 892 µs, total: 86.8 ms
Wall time: 73.2 ms


### Image Redaction - Exercise
Now let's practice what you have learned.

We are going to a variation. Rather than redact the face, let's redact the areas within the eyes.  

In [15]:
%%time
import cv2
import ipywidgets
from IPython.display import display

# Reading an image using OpenCV
image = cv2.imread('/content/lena_color.png')
image_widget = ipywidgets.Image(format='png')

# Load the pre-trained face and eye cascade classifiers
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Converting BGR image into an RGB image
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Detect faces in the image
faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

# Loop over the detected faces
for (x, y, w, h) in faces:
    roi_gray = gray[y:y + h, x:x + w]
    roi_color = image_rgb[y:y + h, x:x + w]

    # Detect eyes within the face region
    eyes = eye_cascade.detectMultiScale(roi_gray)

    # Apply Gaussian blur to each detected eye
    for (ex, ey, ew, eh) in eyes:
        eye_roi = roi_color[ey:ey + eh, ex:ex + ew]
        blurred_eye = cv2.GaussianBlur(eye_roi, (23, 23), 30)
        roi_color[ey:ey + eh, ex:ex + ew] = blurred_eye

# Display the output
image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)
image_widget.value = bytes(cv2.imencode('.png', image_bgr)[1])
display(image_widget)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x00\x00\x00\x01\x00\x08\x02\x00\x00\x00\xd3\x10?…

CPU times: user 110 ms, sys: 4.57 ms, total: 115 ms
Wall time: 161 ms
